In [1]:
import pandas as pd
import numpy as np
import os
import cPickle as pkl
import h5py

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data_dir = "./data/clusterdata-2011-2/"

task_events_header = ['timestamp','missing_info', 'job_id','task_index','machine_id','event_type',
                      'user_name','sched_class','priority','cpu_req','ram_req','space_req','diff_machine']

machine_events_header = ['timestamp','machine_id','event_type','platform_id','cpu_cap','mem_cap']

task_usage_header = ['start_time','end_time','job_id','task_index','machine_id', 'mean_cpu_usage',
                     'canon_memory_usage', 'assign_memory_usage', 'unmapped_cache', 'total_cache', 
                     'max_mem', 'mean_io', 'mean_space', 'max_cpu', 'max_io', 'cpi', 'mai', 'sample',
                     'agg_type', 'sample_cpu_usage']

In [4]:
df = pd.read_csv(data_dir+"task_events/part-00000-of-00500.csv.gz",header=None)
# df.columns = task_events_header

In [5]:
# df[df['job_id'] == 3418309]
df['event_type'].value_counts()


0    201491
1    189994
4     43237
5     10505
2      4338
3       581
Name: event_type, dtype: int64

# Machines that are removed from the cluster were loaded more than X times their mean resource usage.

In [ ]:
first_stamp = pd.read_csv(data_dir+"task_usage/part-00000-of-00500.csv.gz",header=None)
mid_stamp = pd.read_csv(data_dir+"task_usage/part-00050-of-00500.csv.gz",header=None)
last_stamp = pd.read_csv(data_dir+"task_usage/part-00100-of-00500.csv.gz",header=None)
first_stamp.columns = task_usage_header
mid_stamp.columns = task_usage_header
last_stamp.columns = task_usage_header
start_time = int(first_stamp.iloc[0]['start_time']/1000000)
mid_time = int(mid_stamp.iloc[-1]['start_time']/1000000)
end_time = int(last_stamp.iloc[-1]['start_time']/1000000)

del first_stamp
del mid_stamp
del last_stamp

In [ ]:
hf = h5py.File('./data/time_stamps.h5','w')
hf.create_dataset('time',data=[start_time,mid_time,end_time])
hf.close()

### Get Machine IDs of all machines in the first 100 task usage files

In [ ]:
all_machines = np.array([])
for i in xrange(101):
    print i
    df = pd.read_csv(data_dir + "task_usage/part-%05d-of-00500.csv.gz" % i,header=None)
    df.columns = task_usage_header
    m = df.groupby('machine_id').size().reset_index(name='counts')
    m_count = m[m['counts'] > 100]['machine_id']
    all_machines = np.append(all_machines, m_count.values)

all_machines = np.unique(all_machines)

h5 = h5py.File('./data/machines_in_100','w')
h5.create_dataset('ids', data=list(all_machines))
h5.close()

In [12]:
hf = h5py.File('./data/machines_in_100','r')
all_machines = np.array(hf['ids'])
hf.close()

hf = h5py.File('./data/time_stamps.h5','r')
ts = np.array(hf['time'])
start_time = ts[0]
hf.close()



cols = range(start_time,end_time)
machine_usage = pd.DataFrame(data=np.empty((len(all_machines),len(cols))), columns=cols,dtype=np.float16)
machine_usage.index = np.array(all_machines)
machine_usage.replace(0,np.nan,inplace=True)

TypeError: int() argument must be a string or a number, not 'Dataset'

In [11]:
hf.close()

In [124]:
for index,row in df.iterrows():
    if index > 2:
        break
    print row['start_time']/1000000

501720.0
501720.0
501720.0
